In [2]:
import pylab as pl # linear algebra + plots
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import gc
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict, Counter
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
from scipy.stats import pearsonr
from scipy.sparse import hstack
from multiprocessing import Pool

Ttr = pd.read_csv('Data/train.csv')
Tts = pd.read_csv('Data/test.csv', low_memory=False)
R = pd.read_csv('Data/resources.csv')

In [3]:
# combine the tables into one
target = 'project_is_approved'
Ttr['tr'] = 1; Tts['tr'] = 0
Ttr['ts'] = 0; Tts['ts'] = 1

T = pd.concat((Ttr,Tts))

T.loc[T.project_essay_4.isnull(), ['project_essay_4','project_essay_2']] = \
    T.loc[T.project_essay_4.isnull(), ['project_essay_2','project_essay_4']].values

T[['project_essay_2','project_essay_3']] = T[['project_essay_2','project_essay_3']].fillna('')

T['project_essay_1'] = T.apply(lambda row: ' '.join([str(row['project_essay_1']), 
                                                     str(row['project_essay_2'])]), axis=1)
T['project_essay_2'] = T.apply(lambda row: ' '.join([str(row['project_essay_3']),
                                                     str(row['project_essay_4'])]), axis=1)

T = T.drop(['project_essay_3', 'project_essay_4'], axis=1)


In [4]:
R['priceAll'] = R['quantity']*R['price']
newR = R.groupby('id').agg({'description':'count',
                            'quantity':'sum',
                            'price':'sum',
                            'priceAll':'sum'}).rename(columns={'description':'items'})
newR['avgPrice'] = newR.priceAll / newR.quantity
numFeatures = ['items', 'quantity', 'price', 'priceAll', 'avgPrice']

for func in ['min', 'max', 'mean']:
    newR = newR.join(R.groupby('id').agg({'quantity':func,
                                          'price':func,
                                          'priceAll':func}).rename(
                                columns={'quantity':func+'Quantity',
                                         'price':func+'Price',
                                         'priceAll':func+'PriceAll'}).fillna(0))
    numFeatures += [func+'Quantity', func+'Price', func+'PriceAll']

newR = newR.join(R.groupby('id').agg({'description':lambda x:' '.join(x.values.astype(str))}).rename(
    columns={'description':'resource_description'}))

T = T.join(newR, on='id')

del Ttr, Tts, R, newR
gc.collect();

In [5]:
le = LabelEncoder()
T['teacher_id'] = le.fit_transform(T['teacher_id'])
numFeatures += ['teacher_number_of_previously_posted_projects','teacher_id']

statFeatures = []
for col in ['school_state', 'teacher_id', 'teacher_prefix', 'project_grade_category', 'project_subject_categories', 'project_subject_subcategories', 'teacher_number_of_previously_posted_projects']:
    Stat = T[['id', col]].groupby(col).agg('count').rename(columns={'id':col+'_stat'})
    Stat /= Stat.sum()
    T = T.join(Stat, on=col)
    statFeatures.append(col+'_stat')

In [7]:
%%time
textColumns = ['project_essay_1', 'project_essay_2', 'project_resource_summary', 'resource_description', 'project_title']

def getSentFeat(s):
    sent = TextBlob(s).sentiment
    return (sent.polarity, sent.subjectivity)

print('sentimental analysis')


print('key words')
KeyChars = ['!', '\?', '@', '#', '\$', '%', '&', '\*', '\(', '\[', '\{', '\|', '-', '_', '=', '\+',
            '\.', ':', ';', ',', '/', '\\\\r', '\\\\t', '\\"', '\.\.\.', 'etc', 'http']
for col in textColumns:
    for c in KeyChars:
        T[col+'_'+c] = T[col].apply(lambda x: len(re.findall(c, x.lower())))
        numFeatures.append(col+'_'+c)

#####
print('num words')
for col in textColumns:
    T['n_'+col] = T[col].apply(lambda x: len(x.split()))
    numFeatures.append('n_'+col)

#####
print('word tags')
Tags = ['CC', 'CD', 'DT', 'IN', 'JJ', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 
        'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 
        'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
def getTagFeat(s):
    d = Counter([t[1] for t in pos_tag(s.split())])
    return [d[t] for t in Tags]

with Pool(4) as p:
    for col in textColumns:
        temp = pl.array(list(p.map(getTagFeat, T[col])))
        for i, t in enumerate(Tags):
            if temp[:,i].sum() == 0:
                continue
            T[col+'_'+t] = temp[:, i]
            numFeatures += [col+'_'+t]

#####
print('common words')
for i, col1 in enumerate(textColumns[:-1]):
    for col2 in textColumns[i+1:]:
        T['%s_%s_common' % (col1, col2)] = T.apply(lambda row:len(set(re.split('\W', row[col1])).intersection(re.split('\W', row[col2]))), axis=1)
        numFeatures.append('%s_%s_common' % (col1, col2))



sentimental analysis
key words
num words
word tags
common words
CPU times: user 8min 23s, sys: 1min 1s, total: 9min 25s
Wall time: 1h 5min 56s


In [9]:
dateCol = 'project_submitted_datetime'
def getTimeFeatures(T):
    T['year'] = T[dateCol].apply(lambda x: x.year)
    T['month'] = T[dateCol].apply(lambda x: x.month)
    T['day'] = T[dateCol].apply(lambda x: x.day)
    T['dow'] = T[dateCol].apply(lambda x: x.dayofweek)
    T['hour'] = T[dateCol].apply(lambda x: x.hour)
    T['days'] = (T[dateCol]-T[dateCol].min()).apply(lambda x: x.days)
    return T

T[dateCol] = pd.to_datetime(T[dateCol])
T = getTimeFeatures(T)

P_tar = T[T.tr==1][target].mean()
timeFeatures = ['year', 'month', 'day', 'dow', 'hour', 'days']
for col in timeFeatures:
    Stat = T[['id', col]].groupby(col).agg('count').rename(columns={'id':col+'_stat'})
    Stat /= Stat.sum()
    T = T.join(Stat, on=col)
    statFeatures.append(col+'_stat')

numFeatures += timeFeatures
numFeatures += statFeatures

In [10]:
%%time
T2 = T[numFeatures+['id','tr','ts',target]].copy()
Ttr = T2[T.tr==1]
Tar_tr = Ttr[target].values
n = 10
inx = [pl.randint(0, Ttr.shape[0], int(Ttr.shape[0]/n)) for k in range(n)]
# inx is used for crossvalidation of calculating the correlation and p-value
Corr = {}
for c in numFeatures:
    # since some values might be 0s, I use x+1 to avoid missing some important relations
    C1,P1=pl.nanmean([pearsonr(Tar_tr[inx[k]],   (1+Ttr[c].iloc[inx[k]])) for k in range(n)], 0)
    C2,P2=pl.nanmean([pearsonr(Tar_tr[inx[k]], 1/(1+Ttr[c].iloc[inx[k]])) for k in range(n)], 0)
    if P2<P1:
        T2[c] = 1/(1+T2[c])
        Corr[c] = [C2,P2]
    else:
        T2[c] = 1+T2[c]
        Corr[c] = [C1,P1]

polyCol = []
thrP = 0.01
thrC = 0.02
print('columns \t\t\t Corr1 \t\t Corr2 \t\t Corr Combined')
for i, c1 in enumerate(numFeatures[:-1]):
    C1, P1 = Corr[c1]
    for c2 in numFeatures[i+1:]:
        C2, P2 = Corr[c2]
        V = T2[c1] * T2[c2]
        Vtr = V[T2.tr==1].values
        C, P = pl.nanmean([pearsonr(Tar_tr[inx[k]], Vtr[inx[k]]) for k in range(n)], 0)
        if P<thrP and abs(C) - max(abs(C1),abs(C2)) > thrC:
            T[c1+'_'+c2+'_poly'] = V
            polyCol.append(c1+'_'+c2+'_poly')
            print(c1+'_'+c2, '\t\t(%g, %g)\t(%g, %g)\t(%g, %g)'%(C1,P1, C2,P2, C,P))

numFeatures += polyCol
print(len(numFeatures))
del T2, Ttr
gc.collect();

/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:3000: RuntimeWarning: invalid value encountered in subtract
  xm, ym = x - mx, y - my
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:3001: RuntimeWarning: invalid value encountered in reduce
  r_num = np.add.reduce(xm * ym)
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


columns 			 Corr1 		 Corr2 		 Corr Combined
quantity_price 		(0.1133, 2.52442e-48)	(0.0825278, 8.86382e-25)	(0.141463, 1.54238e-73)
price_avgPrice 		(0.0825278, 8.86382e-25)	(0.0529284, 9.61872e-11)	(0.194873, 1.93954e-147)
price_maxQuantity 		(0.0825278, 8.86382e-25)	(0.0171186, 0.102521)	(0.109452, 1.30994e-41)
price_meanPrice 		(0.0825278, 8.86382e-25)	(0.0494913, 9.59655e-10)	(0.196091, 6.29086e-150)
priceAll_avgPrice 		(0.0482872, 1.61269e-07)	(0.0529284, 9.61872e-11)	(0.108635, 4.9114e-44)
priceAll_maxPriceAll 		(0.0482872, 1.61269e-07)	(0.0461969, 2.41348e-07)	(0.174737, 1.00636e-114)
priceAll_meanPrice 		(0.0482872, 1.61269e-07)	(0.0494913, 9.59655e-10)	(0.10563, 3.27763e-42)
priceAll_meanPriceAll 		(0.0482872, 1.61269e-07)	(0.0813926, 7.79207e-24)	(0.195413, 6.2407e-149)
avgPrice_maxPrice 		(0.0529284, 9.61872e-11)	(0.059798, 1.51808e-13)	(0.196227, 6.09872e-145)
minQuantity_n_project_essay_2 		(-0.135848, 8.13501e-67)	(-0.093459, 1.09641e-29)	(-0.171322, 1.93966e-113)
minQuan

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: Mean of empty slice


resource_description_\(_project_resource_summary_NNS 		(0.0525535, 1.19256e-06)	(0.052544, 4.53083e-09)	(0.0730806, 1.77224e-14)
resource_description_-_n_project_essay_2 		(-0.150014, 7.65456e-81)	(-0.093459, 1.09641e-29)	(-0.173629, 9.29776e-103)
resource_description_-_project_essay_2_NN 		(-0.150014, 7.65456e-81)	(-0.0847483, 1.41461e-24)	(-0.177702, 2.13644e-111)
resource_description_\._resource_description_NNS 		(0.0909247, 2.39148e-27)	(0.0890114, 2.15641e-25)	(0.112269, 1.89905e-42)
resource_description_,_project_essay_2_NN 		(-0.0900705, 6.14321e-24)	(-0.0847483, 1.41461e-24)	(-0.114858, 1.3584e-41)
resource_description_,_project_essay_2_project_resource_summary_common 		(-0.0900705, 6.14321e-24)	(-0.0734123, 1.08089e-16)	(-0.110717, 9.40602e-37)
resource_description_/_project_essay_2_DT 		(0.0730306, 1.00223e-17)	(0.0681042, 3.8943e-14)	(0.0969172, 1.94542e-32)
resource_description_/_project_essay_2_IN 		(0.0730306, 1.00223e-17)	(0.0655694, 3.87812e-16)	(0.0944826, 8.17636e-33)

In [11]:
T.to_csv('T_XT.csv',index = False)